<a href="https://colab.research.google.com/github/Talha1818/Vector-Database-Management-System-Word-Emdedding-ChromaDB/blob/master/LangChain_ChromaDB_ChatGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install langchain pypdf sentence_transformers InstructorEmbedding
!pip install chromadb==0.3.26 -q
!pip show langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.3/270.3 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 78.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 72.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 9.3 MB/s eta 0:00:00

In [2]:
import pandas as pd
import numpy as np
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders.csv_loader import CSVLoader


from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

/usr/local/lib/python3.10/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


In [3]:
file_path = "/content/drive/MyDrive/Annotation Folder/Final_Text_Dataset_30_May_2023.csv"
df_ = pd.read_csv(file_path, nrows=1000)[['text', 'token']]
# df_ = df[df['token'].isin([15, 14, 85,100,77,116,90])][['text', 'token']].reset_index(drop=True)
df_.shape

(1000, 2)

In [4]:
df_.head()

,text,token
0,"HADDONEIFLD - CITY ABSTRACT, L 1LBS 10F1 |\nMa...",15
1,"City Abstract, LLC ALTA Buyer's Settlement Sta...",11
2,Peseription\nTitle - Notary Fee to City Abstra...,11
3,"City Abstract, LLC ALTA Seller's Settlement St...",13
4,|\n| Acknowledgement\n| We/I have carefully re...,13


In [5]:
df_['text'][0]

'HADDONEIFLD - CITY ABSTRACT, L 1LBS 10F1 |\\nManta aE ow 201\\n|\\nS \\u2018O;\\nHT 565068858\\nGREENTREE MORTGAGE COMPANY, L.P. |\\n2 EASTWICK DRIVE\\nSUITE 300 |\\nGIBBSBORO NJ 08026\\n|\\n|\\n|\\nBR NJ 081 9-01\\nL}\\nHAN\\n% \\xa9Ez |\\nsoe Rk $, |\\nSees\\nUPS GROUND |\\nTRACKING #: 1Z 2F6 793 03 9794 0674 |\\nBILLING: P/P\\nXOL 22.10.12 NV45 43.0A 10/2022*\\n|\\n|\\n\\x0c'

In [6]:
import re

 # Define cleaning function
def clean_text(text):
  text = text.replace("/", " ")
  cleaned_text = re.sub(r'[^\w\s]', '', text)
  cleaned_text = re.sub(r'[\n\r]', ' ', cleaned_text)
  cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
  return cleaned_text

# Original Text
text = "HADDONEIFLD - CITY ABSTRACT, L 1LBS 10F1 |\nManta aE ow 201\n|\nS \u2018O;\nHT 565068858\nGREENTREE MORTGAGE COMPANY, L.P. |\n2 EASTWICK DRIVE\nSUITE 300 |\nGIBBSBORO NJ 08026\n|\n|\n|\nBR NJ 081 9-01\nL}\nHAN\n% \xa9Ez |\nsoe Rk $, |\nSees\nUPS GROUND |\nTRACKING #: 1Z 2F6 793 03 9794 0674 |\nBILLING: P/P\nXOL 22.10.12 NV45 43.0A 10/2022*\n|\n|\n\x0c"

# Cleaned Text
cleaned_text = clean_text(text)
print(cleaned_text)


HADDONEIFLD CITY ABSTRACT L 1LBS 10F1 Manta aE ow 201 S O HT 565068858 GREENTREE MORTGAGE COMPANY LP 2 EASTWICK DRIVE SUITE 300 GIBBSBORO NJ 08026 BR NJ 081 901 L HAN Ez soe Rk Sees UPS GROUND TRACKING 1Z 2F6 793 03 9794 0674 BILLING P P XOL 221012 NV45 430A 10 2022


In [ ]:
cleaned_text

'HADDONEIFLD CITY ABSTRACT L 1LBS 10F1 Manta aE ow 201 S O HT 565068858 GREENTREE MORTGAGE COMPANY LP 2 EASTWICK DRIVE SUITE 300 GIBBSBORO NJ 08026 BR NJ 081 901 L HAN Ez soe Rk Sees UPS GROUND TRACKING 1Z 2F6 793 03 9794 0674 BILLING P P XOL 221012 NV45 430A 10 2022'

In [ ]:
from tqdm.auto import tqdm
tqdm.pandas(desc='cleaning')
df_['text'] = df_['text'].progress_apply(clean_text)

cleaning:   0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
df_.head()

,text,token
0,HADDONEIFLD CITY ABSTRACT L 1LBS 10F1 nManta a...,15
1,City Abstract LLC ALTA Buyers Settlement State...,11
2,PeseriptionnTitle Notary Fee to City Abstract ...,11
3,City Abstract LLC ALTA Sellers Settlement Stat...,13
4,n Acknowledgementn We I have carefully reviewe...,13


In [ ]:
df_.to_csv("data.csv", index=False)

In [ ]:
loader = CSVLoader(file_path='/content/data.csv')
documents = loader.load()

In [ ]:
documents[0]

Document(page_content='text: HADDONEIFLD CITY ABSTRACT L 1LBS 10F1 nManta aE ow 201nnS u2018OnHT 565068858nGREENTREE MORTGAGE COMPANY LP n2 EASTWICK DRIVEnSUITE 300 nGIBBSBORO NJ 08026nnnnBR NJ 081 901nLnHANn xa9Ez nsoe Rk nSeesnUPS GROUND nTRACKING 1Z 2F6 793 03 9794 0674 nBILLING P PnXOL 221012 NV45 430A 10 2022nnnx0c\ntoken: 15', metadata={'source': '/content/data.csv', 'row': 0})

In [ ]:
#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [ ]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",
                                                      model_kwargs={"device": "cuda"})

load INSTRUCTOR_Transformer
max_seq_length  512


In [ ]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'

## Here is the nmew embeddings being used
embedding = instructor_embeddings

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

In [ ]:
# persiste the db to disk
vectordb.persist()
vectordb = None

In [ ]:
# Now we can load the persisted database from disk, and use it as normal.
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding)

In [ ]:
retriever = vectordb.as_retriever()

In [ ]:
docs = retriever.get_relevant_documents("What is the NFM Lending?")

In [ ]:
docs[0]

Document(page_content='text: ee xb0 NFM Inc dba NFM LendingnSs NFM Lending A ia ic A Company NMLS 2893nFanily of Lancers sme SE ttc a 1524 Kings Highwayn Swedesboro NJ 08085nMLO NMLS 95005nUNDISCLOSED DEBT ACKNOWLEDGEMENTnLoan Number 23921040557nBorrowers Matthew ReschnProperty Address235 W Thompson StnCity State and Zip Philadelphia PA 19122nThank you for Choosing NFM Inc dba NFM Lending for your mortgage financing needs It isnimportant that we have an accurate representation of your financial commitments so we canndetermine if you qualify for your loan Each time someone requests your credit report andninquiry is noted on the report The most common reason this occurs is in connection with annapplication for credit such as a mortgage loan auto loan credit card etcnWe will continually monitor your credit activity during the loan application process andnmay obtain a new credit report prior to close New accounts and inquires my impact yournloan approval or loan closing as they can represe

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

In [ ]:
retriever.search_type

'similarity'

## **Deleteing the DB**

In [ ]:
import os

# Define the directory path
directory = "/content/drive/MyDrive/LangChainDatabase"

# Check if the directory exists
if not os.path.exists(directory):
    # If it doesn't exist, create the directory
    os.makedirs(directory)
    print(f"Directory '{directory}' created successfully!")
else:
    print(f"Directory '{directory}' already exists.")


Directory '/content/drive/MyDrive/LangChainDatabase' created successfully!


In [ ]:
!zip -r /content/drive/MyDrive/LangChainDatabase/db.zip ./db

  adding: db/ (stored 0%)
  adding: db/index/ (stored 0%)
  adding: db/index/index_26b2cac7-8f03-40eb-b59a-0f396ab00ce4.bin (deflated 18%)
  adding: db/index/uuid_to_id_26b2cac7-8f03-40eb-b59a-0f396ab00ce4.pkl (deflated 40%)
  adding: db/index/index_metadata_26b2cac7-8f03-40eb-b59a-0f396ab00ce4.pkl (deflated 14%)
  adding: db/index/id_to_uuid_26b2cac7-8f03-40eb-b59a-0f396ab00ce4.pkl (deflated 36%)
  adding: db/chroma-embeddings.parquet (deflated 20%)
  adding: db/chroma-collections.parquet (deflated 50%)


In [ ]:
# To cleanup, you can delete the collection
vectordb.delete_collection()
vectordb.persist()

# delete the directory
!rm -rf db/

## **Starting again loading the db**

In [ ]:
!unzip /content/drive/MyDrive/LangChainDatabase/db.zip

Archive:  /content/drive/MyDrive/LangChainDatabase/db.zip
   creating: db/
   creating: db/index/
  inflating: db/index/index_26b2cac7-8f03-40eb-b59a-0f396ab00ce4.bin  
  inflating: db/index/uuid_to_id_26b2cac7-8f03-40eb-b59a-0f396ab00ce4.pkl  
  inflating: db/index/index_metadata_26b2cac7-8f03-40eb-b59a-0f396ab00ce4.pkl  
  inflating: db/index/id_to_uuid_26b2cac7-8f03-40eb-b59a-0f396ab00ce4.pkl  
  inflating: db/chroma-embeddings.parquet  
  inflating: db/chroma-collections.parquet  


In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

In [ ]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",
                                                      model_kwargs={"device": "cuda"})


load INSTRUCTOR_Transformer
max_seq_length  512


In [ ]:
persist_directory = 'db'
embedding = instructor_embeddings

vectordb2 = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding,
                   )

retriever = vectordb2.as_retriever(search_kwargs={"k": 3})

In [ ]:
docs = retriever.get_relevant_documents("Eleborate Right to Financial Privacy Act?")
docs[0]

Document(page_content='rejected If younbelieve that your financial records have been disclosed improperly you may have legal rights under the Right to FinancialnPrivacy Act of 1978 12 USCS Sections 3401 et seqnUg cfr 4 2 2022nMATTHEW ALEXANDER RESCH DATEnICE Mortgage Technology Inc GBCT 0719nGBCT CLSn08 09 2022 0933 AM PSTnnx0c', metadata={'source': '/content/data.csv', 'row': 603})

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""


## **Make a chain**

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 2.1 MB/s eta 0:00:00


In [ ]:
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(),
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [ ]:
## Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [ ]:
## Cite sources

import textwrap

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['result']))
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [ ]:
# full example
query = "What is the LLC?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 The LLC is a limited liability company.


Sources:
/content/data.csv
/content/data.csv
/content/data.csv


In [ ]:
# full example
query = "What is mean by mortgage documents?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Mortgage documents are legal documents that provide information about a loan agreement between a lender and a
borrower. They include details about the loan amount, interest rate, repayment schedule, and other relevant
information.


Sources:
/content/data.csv
/content/data.csv
/content/data.csv


In [ ]:
# full example
query = "What is the criteria to buy new home, which things we need for?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 To buy a new home, you will need to provide your current financial information, such as income, debts,
expenses, and credit obligations. You will also need to provide information about your ethnicity, sex, and
race in order to comply with federal laws related to equal credit opportunity and fair housing.


Sources:
/content/data.csv
/content/data.csv
/content/data.csv


In [ ]:
# full example
query = "How much total cost to buy a new home?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 The total cost to buy a new home would depend on the price of the home and the closing costs associated with
it. In the example given, the sale price of the property is $17,650,000 and the closing costs paid at closing
are $2,119,694, for a total of $19,769,694.


Sources:
/content/data.csv
/content/data.csv
/content/data.csv
